# 前処理

- `madb`からcsv形式でデータを取得する
- `pytrend`で各漫画作品のGoogle検索量を取得

## 環境構築

In [1]:
# Notebook初期設定
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import glob
import json
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import zipfile

In [3]:
DIR_IN = '../madb/data/json-ld'
DIR_TMP = '../data/preprocess/tmp'
DIR_OUT = '../data/preprocess/out'

In [4]:
FNS_CM = [
    'cm102',
    'cm105',
    'cm106',
]

In [5]:
# CM105で使用するカラム
COL_CM105 = [
    'identifier',
    'label',
    'note',
    'publicationPeriodicity',
    'name',
    'publisher',
    'dayPublishedFinal',
]

In [6]:
!ls {DIR_IN}

metadata_an-col_an207_json.zip	 metadata_cm-item_cm101_json.zip
metadata_an-col_an208_json.zip	 metadata_cm-item_cm102_json.zip
metadata_an-col_an210_json.zip	 metadata_cm-item_cm103_json.zip
metadata_an-item_an201_json.zip  metadata_co-curate_co504_json.zip
metadata_an-item_an202_json.zip  metadata_gm-col_gm305_json.zip
metadata_an-item_an205_json.zip  metadata_gm-col_gm306_json.zip
metadata_cm-col_cm104_json.zip	 metadata_gm-item_gm301_json.zip
metadata_cm-col_cm105_json.zip	 metadata_ma-col_ma408_json.zip
metadata_cm-col_cm106_json.zip	 metadata_ma-item_ma401_json.zip


## 関数

In [7]:
def read_json(path):
    """
    jsonファイルを辞書として読み込む関数．

    Params:
        path (str): 読込対象ファイルパス
    Returns:
        dict: 辞書
    """
    with open(path, 'r') as f:
        dct = json.load(f)

    return dct

## 解凍

マンガ系のデータ（`*cm*`）のみ`DIR_TMP`に解凍する．

In [22]:
ps_cm = glob.glob(f'{DIR_IN}/*_cm*')

In [23]:
for p_from in tqdm(ps_cm):
    p_to = p_from.replace(DIR_IN, DIR_TMP).replace('.zip', '')
    
    with zipfile.ZipFile(p_from) as z:
        z.extractall(p_to)

  0%|          | 0/6 [00:00<?, ?it/s]

## 前処理

In [8]:
ps_cm = {cm: glob.glob(f'{DIR_TMP}/*{cm}*/*') for cm in FNS_CM}

In [9]:
ps_cm

{'cm102': ['../data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00001.json',
  '../data/preprocess/tmp/metadata_cm-item_cm102_json/metadata_cm-item_cm102_json\\metadata_cm-item_cm102_00002.json'],
 'cm105': ['../data/preprocess/tmp/metadata_cm-col_cm105_json/metadata_cm-col_cm105_json\\metadata_cm-col_cm105_00001.json'],
 'cm106': ['../data/preprocess/tmp/metadata_cm-col_cm106_json/metadata_cm-col_cm106_json\\metadata_cm-col_cm106_00001.json']}

### `cm105`

In [12]:
cm105 = read_json(ps_cm['cm105'][0])

In [22]:
df_cm105 = pd.DataFrame(cm105['@graph'])[COL_CM105]

In [24]:
def get_published_name(name):
    """nameからpublished_nameを取得"""
    return name[0]

In [26]:
def get_furigana(name):
    """nameから振り仮名を取得"""
    return name[1]['@value']

In [27]:
df_cm105['mcname'] = df_cm105['name'].apply(lambda x: get_published_name(x))

In [28]:
df_cm105

,identifier,label,note,publicationPeriodicity,name,publisher,dayPublishedFinal,mcname
0,C117451,斬鬼,・『ヤングキングSPECIAL増刊 斬鬼』から独立創刊。 ／ ・通巻号数は『ヤングキングSP...,隔月刊,"[斬鬼, {'@language': 'ja-hrkt', '@value': 'ザンキ'}]",少年画報社,NaN,斬鬼
1,C117452,月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL,NaN,不定期刊 （no determinable frequency）,"[月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL, {'@lan...",集英社 ∥ シュウエイシャ,NaN,月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL
2,C117453,YOUNG YOU Special,[編]集英社 ／ 「Young you」の増刊,NaN,"[{'@language': 'ja-hrkt', '@value': 'ヤングユースペシャ...",集英社,NaN,"{'@language': 'ja-hrkt', '@value': 'ヤングユースペシャル..."
3,C117455,YOUNG YOU,『月刊セブンティーン特別編集 ヤングユー』 - 『ヤングユー』独立創刊 ／ ・『月刊セブンテ...,月刊 （monthly）,"[{'@language': 'ja-hrkt', '@value': 'ヤングユー'}, ...",集英社 ∥ シュウエイシャ,2005-11-01,"{'@language': 'ja-hrkt', '@value': 'ヤングユー'}"
4,C117456,別冊YOUNG YOU,"別冊 YOUNG YOU ／ 刊行頻度,大きさの変更あり 増刊とも ／ 1987年4月30日...",隔月刊,"[{'@language': 'ja-hrkt', '@value': 'ベッサツヤングユー...",集英社 ∥ シュウエイシャ,2002-12-30,"{'@language': 'ja-hrkt', '@value': 'ベッサツヤングユー,..."
...,...,...,...,...,...,...,...,...
5723,C123543,comicアンスリウム,・『DMM』の増刊より独立。,NaN,"[comicアンスリウム, {'@language': 'ja-hrkt', '@value...",ジーオーティー,NaN,comicアンスリウム
5724,C123544,月刊ASUKAファンタジーDX増刊,『月刊ASUKAファンタジーDX』の増刊,NaN,"[{'@language': 'ja-hrkt', '@value': 'ゲッカンアスカファ...",角川書店,NaN,"{'@language': 'ja-hrkt', '@value': 'ゲッカンアスカファン..."
5725,C123545,少年ビッグコミック特別編集,『少年ビッグコミック』の『ビッグコミック』の雑誌コードがあり、かつISBNのあるものもある,NaN,"[{'@language': 'ja-hrkt', '@value': 'ショウネンビッグコ...",小学館,NaN,"{'@language': 'ja-hrkt', '@value': 'ショウネンビッグコミ..."
5726,C123546,いつかどこかで増刊,1977年3月に、季刊から隔月刊へ変わる際に刊行,NaN,"[いつかどこかで増刊, {'@language': 'ja-hrkt', '@value':...",立風書房,NaN,いつかどこかで増刊


In [16]:
pd.DataFrame(cm105['@graph']).head().T

,0,1,2,3,4
@id,https://mediaarts-db.bunka.go.jp/id/C117451,https://mediaarts-db.bunka.go.jp/id/C117452,https://mediaarts-db.bunka.go.jp/id/C117453,https://mediaarts-db.bunka.go.jp/id/C117455,https://mediaarts-db.bunka.go.jp/id/C117456
rdf:type,class:Collection,class:Collection,class:Collection,class:Collection,class:Collection
label,斬鬼,月刊少年ジャンプ 週刊ヤングジャンプ共同編集 弓月光ジャンプSPECIAL,YOUNG YOU Special,YOUNG YOU,別冊YOUNG YOU
identifier,C117451,C117452,C117453,C117455,C117456
additionalType,class:CM,class:CM,class:CM,class:CM,class:CM
genre,雑誌全号,雑誌全号,雑誌全号,雑誌全号,雑誌全号
mtid,ZANKrgl,YZKSjmp,YYOUspe,YYOUrgl,YYOUoth
oldDateModified,2013-11-13T11:48:02,2017-02-08T11:33:24,2013-11-13T11:48:02,2018-08-15T01:43:26,2015-03-15T16:30:15
note,・『ヤングキングSPECIAL増刊 斬鬼』から独立創刊。 ／ ・通巻号数は『ヤングキングSP...,NaN,[編]集英社 ／ 「Young you」の増刊,『月刊セブンティーン特別編集 ヤングユー』 - 『ヤングユー』独立創刊 ／ ・『月刊セブンテ...,"別冊 YOUNG YOU ／ 刊行頻度,大きさの変更あり 増刊とも ／ 1987年4月30日..."
publicationPeriodicity,隔月刊,不定期刊 （no determinable frequency）,NaN,月刊 （monthly）,隔月刊


In [17]:
COL_CM105 = [
    'identifier',
    'label',
    'note',
    'publicationPeriodicity',
    'name',
    'publisher',
    'dayPublishedFinal',
]